In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [36]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

def compute_cls_metrics(events_recs_for_binary_metrics):
    
    groupper = events_recs_for_binary_metrics.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum() + groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

In [4]:
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

In [5]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [6]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

In [14]:
import scipy
import sklearn.preprocessing

items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr 

# В genres_csr_col_idx добавляются индексы, соответствующие жанрам.

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

print('item_id_enc max - ', events_train["item_id_enc"].max())

def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v["genre_and_votes"]
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

/tmp/ipykernel_3577/1296505728.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_3577/1296505728.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_3577/1296505728.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

item_id_enc max -  43304


In [15]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [17]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True

/tmp/ipykernel_3577/210734134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["read"] = True


In [18]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()

# считаем количество уникальных пользователей в events_labels
print(events_labels["user_id"].nunique())

99849


In [19]:
# загружаем рекомендации от двух базовых генераторов
import pandas as pd 

als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer")

len(candidates)

82993094

In [20]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], 
                                on=["user_id", "item_id"],
                                how="left"
                            )
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    # все позитивные примеры
    candidates_to_sample.query("target == 1"),
    # по 4 отрицательных на пользователя
    candidates_to_sample.query("target == 0")
        .groupby("user_id")
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
]).reset_index(drop=True)

print(len(candidates_for_train))

213708


In [21]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 72.6ms	remaining: 1m 12s
100:	learn: 0.5118959	total: 1.9s	remaining: 16.9s
200:	learn: 0.5111710	total: 3.81s	remaining: 15.1s
300:	learn: 0.5105208	total: 5.74s	remaining: 13.3s
400:	learn: 0.5100174	total: 7.66s	remaining: 11.4s
500:	learn: 0.5095747	total: 9.6s	remaining: 9.56s
600:	learn: 0.5091600	total: 11.5s	remaining: 7.66s
700:	learn: 0.5087803	total: 13.5s	remaining: 5.74s
800:	learn: 0.5084220	total: 15.4s	remaining: 3.81s
900:	learn: 0.5080930	total: 17.3s	remaining: 1.9s
999:	learn: 0.5078081	total: 19.2s	remaining: 0us


In [22]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]
print(len(candidates_to_rank))

14517152


In [23]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query("rank <= @max_recommendations_per_user")

len(final_recommendations)

7519400

In [24]:
# final_recommendations.to_parquet("final_recommendations_feat.parquet")

In [26]:
items["age"] = 2018 - items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

candidates_for_train = candidates_for_train.merge(
    items[["item_id", "age", "average_rating"]],
    on=["item_id"],
    how="outer"
)
candidates_to_rank = candidates_to_rank.merge(
    items[["item_id", "age", "average_rating"]],
    on=["item_id"],
    how="outer"
)
candidates_to_rank["age"].median()

7.0

In [27]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max() - x.min()).days / 365.25),
        books_read=("item_id", "nunique"),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std")
    )
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features
    
user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")
  
# оставим только тех пользователей, что есть в тесте, для экономии ресурсов
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

user_features_for_ranking = get_user_features(events_inference)

candidates_to_rank = candidates_to_rank.merge(
    user_features_for_ranking, on="user_id", how="left"
)

candidates_for_train["books_read"].median()

32.0

In [28]:
# определяем индексы топ-10 жанров и всех остальных
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column]

# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        # топ жанров
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns),
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)
# убрать всё, что мы уже когда-то навешивали
genre_cols_existing = [c for c in items.columns if c.startswith("genre_")]
if genre_cols_existing:
    items = items.drop(columns=genre_cols_existing)
# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")

def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres
    
user_genres_for_train = get_user_genres(events_train, items, genre_columns)
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left")

romance_id = genres.query("name == 'Romance'").index[0]
romance_col = f"genre_{romance_id}"
candidates_for_train[romance_col].median()

0.038488976462249

In [30]:
from catboost import CatBoostClassifier, Pool

candidates_for_train = candidates_for_train[~candidates_for_train["target"].isna()].copy()
candidates_for_train["target"] = candidates_for_train["target"].astype(int)

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data) 

0:	learn: 0.6467298	total: 27.9ms	remaining: 27.9s
100:	learn: 0.4657506	total: 2.63s	remaining: 23.4s
200:	learn: 0.4575146	total: 5.22s	remaining: 20.7s
300:	learn: 0.4515641	total: 7.83s	remaining: 18.2s
400:	learn: 0.4467942	total: 10.5s	remaining: 15.6s
500:	learn: 0.4423475	total: 13.1s	remaining: 13s
600:	learn: 0.4383226	total: 15.7s	remaining: 10.4s
700:	learn: 0.4347273	total: 18.3s	remaining: 7.79s
800:	learn: 0.4312752	total: 20.9s	remaining: 5.19s
900:	learn: 0.4279893	total: 23.7s	remaining: 2.61s
999:	learn: 0.4249013	total: 26.3s	remaining: 0us


In [31]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставим rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query("rank <= @max_recommendations_per_user") 

In [32]:
final_recommendations.to_parquet("final_recommendations_feat.parquet")

#### **Задание 5 из 6**

Используя отложенную тестовую выборку `events_test_2`, посчитаем метрики `recall` и `precision` для полученных рекомендаций.

In [37]:
# для экономии ресурсов оставим события только тех пользователей, 
# для которых следует оценить рекомендации
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,  # <- тестовая выборка для валидации
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5         # <- считаем precision@5 и recall@5
)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.011, recall: 0.029


### Проверка важности признаков

Любопытно понять, какие признаки вносят наибольший вклад в ранжирование. Алгоритм CatBoost позволяет получить такую информацию (англ. feature importance), которая генерируется во время тренировки модели. Для этого используйте метод `get_feature_importance()`.

#### **Задание 6**

Выполним код для получения информации о важности признаков. Выведем список признаков `feature_importance` в порядке убывания их важности.


In [33]:
feature_importance = pd.DataFrame(
    cb_model.get_feature_importance(),
    index=features,
    columns=["fi"]
)

feature_importance = feature_importance.sort_values("fi", ascending=False)
print(feature_importance)

                       fi
als_score       28.750790
age             18.341400
average_rating  14.320395
books_read       6.095059
reading_years    3.497930
cnt_score        3.134267
genre_18         2.627597
genre_others     2.580376
genre_1          2.570093
genre_25         2.368419
books_per_year   2.285353
genre_34         2.174140
genre_16         1.576704
genre_38         1.569290
genre_33         1.516361
genre_24         1.513510
rating_avg       1.486050
genre_20         1.414986
genre_5          1.162734
rating_std       1.014546


## Алгоритм онлайн-рекомендаций

#### **Шаг 1. Набор похожих объектов**

Чтобы получить набор похожих объектов, можно воспользоваться уже известным алгоритмом ALS из библиотеки `implicit`, у которого на такой случай есть удобный метод `similar_items`.

Воспользуемся им и получим по 10 самых похожих айтемов.

#### **Задание 1 из 6**

Получим набор похожих объектов в `similar_items`.

In [38]:
import scipy
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [39]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train) 

/home/mle-user/mle_projects/mle-recsys-start/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/.venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:00<00:00,  3.62s/it]


In [40]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(),
    "score": sim_item_scores.tolist()
})

similar_items = similar_items.explode(["sim_item_id_enc", "score"])

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype(int)
similar_items["score"] = similar_items["score"].astype(float)

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

Укажем идентификатор объекта, наиболее похожего на объект `7126`.

In [41]:
similar_items.query("item_id_1 == 7126").sort_values("score", ascending=False).head()

,item_id_enc,sim_item_id_enc,score,item_id_1,item_id_2
2352,839,844,0.948725,7126,7190
2352,839,2651,0.940997,7126,24280
2352,839,209,0.930144,7126,1953
2352,839,4916,0.925066,7126,58696
2352,839,3761,0.916340,7126,38296


In [42]:
similar_items.to_parquet("similar_items.parquet")

Можно оценить глазами списки похожих объектов для каких-то уже известных. Создадим для этой цели функцию print_sim_items.

In [45]:
def print_sim_items(item_id, similar_items):
    # Берём только существующие колонки
    possible_cols = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    item_columns_to_use = [c for c in possible_cols if c in items.columns]

    # исходная книга
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)

    # похожие книги
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(
        items[item_columns_to_use].set_index("item_id"),
        left_on="item_id_2",
        right_index=True,
        how="left",
    )
    display(si)

In [46]:
book_ids = [
    7144,   # Ф. М. Достоевский «Преступление и наказание»
    16299,  # Агата Кристи «Десять негритят»
    3,      # Джоан Роулинг «Гарри Поттер и философский камень»
    18135,  # Уильям Шекспир «Ромео и Джульетта»
    17245   # Брэм Стокер «Дракула»
]

for bid in book_ids:
    print(f"\n\n=== Похожие книги для item_id = {bid} ===\n")
    print_sim_items(bid, similar_items)



=== Похожие книги для item_id = 7144 ===



,item_id,author,title,average_rating,ratings_count
840,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,4.19,390293


,item_id_enc,sim_item_id_enc,score,item_id_1,item_id_2,author,title,average_rating,ratings_count
6069,840,1512,0.964479,7144,12505,"Fyodor Dostoyevsky, Anna Brailovsky, Constance...",The Idiot,4.18,76392
6069,840,1566,0.953918,7144,12857,"Fyodor Dostoyevsky, Constance Garnett",The Gambler,3.88,22024
6069,840,5395,0.952009,7144,67326,Fyodor Dostoyevsky,Poor Folk,3.73,4957
6069,840,22683,0.946847,7144,5508624,Leo Tolstoy,Family Happiness,3.85,3337
6069,840,523,0.939762,7144,4934,"Fyodor Dostoyevsky, Fyodor Dostoyevsky, Richar...",The Brothers Karamazov,4.31,158410
6069,840,2180,0.938018,7144,17877,"Fyodor Dostoyevsky, Constance Garnett",The House of the Dead,4.04,8548
6069,840,17144,0.937007,7144,929782,"Jack London, Andrew Sinclair",Martin Eden,4.39,13257
6069,840,2958,0.936360,7144,28382,Nikolai Gogol,Diary of a Madman and Other Stories,4.09,6241
6069,840,2159,0.936320,7144,17690,"Franz Kafka, Max Brod, Willa Muir, Edwin Muir",The Trial,3.98,135862
6069,840,5208,0.934541,7144,63038,Victor Hugo,The Man Who Laughs,4.22,5449




=== Похожие книги для item_id = 16299 ===



,item_id,author,title,average_rating,ratings_count
1991,16299,Agatha Christie,And Then There Were None,4.23,429352


...

,item_id,author,title,average_rating,ratings_count
2203,18135,"William Shakespeare, Paul Werstine, Barbara A....",Romeo and Juliet,3.74,1656919


,item_id_enc,sim_item_id_enc,score,item_id_1,item_id_2,author,title,average_rating,ratings_count
8450,2203,1041,0.920891,18135,8852,William Shakespeare,Macbeth,3.88,502298
8450,2203,167,0.889775,18135,1420,"William Shakespeare, Harold Bloom, Rex Gibson",Hamlet,4.01,526122
8450,2203,904,0.888842,18135,7728,"Sophocles, J.E. Thomas","Antigone (The Theban Plays, #3)",3.61,69075
8450,2203,2120,0.880297,18135,17250,"Arthur Miller, Christopher Bigsby",The Crucible,3.55,247565
8450,2203,188,0.876469,18135,1622,"William Shakespeare, Paul Werstine, Barbara A....",A Midsummer Night's Dream,3.94,340695
8450,2203,1487,0.869903,18135,12296,"Nathaniel Hawthorne, Thomas E. Connolly, Faust...",The Scarlet Letter,3.37,515452
8450,2203,1587,0.866114,18135,13006,"William Shakespeare, Roma Gill",Julius Caesar,3.66,121890
8450,2203,182,0.854540,18135,1554,"Sophocles, J.E. Thomas","Oedipus Rex (The Theban Plays, #1)",3.67,122126
8450,2203,1586,0.848521,18135,12996,William Shakespeare,Othello,3.88,242511
8450,2203,320,0.836064,18135,2956,"Mark Twain, Guy Cardwell, John Seelye, Walter ...",The Adventures of Huckleberry Finn,3.80,969291




=== Похожие книги для item_id = 17245 ===



,item_id,author,title,average_rating,ratings_count
2119,17245,"Bram Stoker, Nina Auerbach, David J. Skal",Dracula,3.98,636895


,item_id_enc,sim_item_id_enc,score,item_id_1,item_id_2,author,title,average_rating,ratings_count
2176,2119,13789,0.928823,17245,480204,"Gaston Leroux, Alexander Teixeira de Mattos",The Phantom of the Opera,3.97,144859
2176,2119,4578,0.900337,17245,51496,"Robert Louis Stevenson, Vladimir Nabokov, Merv...",The Strange Case of Dr. Jekyll and Mr. Hyde,3.79,229898
2176,2119,6654,0.898938,17245,93261,Washington Irving,The Legend of Sleepy Hollow,3.74,26776
2176,2119,54,0.897706,17245,295,Robert Louis Stevenson,Treasure Island,3.82,274424
2176,2119,286,0.896470,17245,2623,"Charles Dickens, Marisa Sestino",Great Expectations,3.75,468462
2176,2119,2211,0.895993,17245,18254,"Charles Dickens, Philip Horne, Gerald Dickens",Oliver Twist,3.85,235560
2176,2119,844,0.886899,17245,7190,Alexandre Dumas,"The Three Musketeers (The D'Artagnan Romances,...",4.06,198892
2176,2119,2646,0.881911,17245,24213,"Lewis Carroll, John Tenniel, Martin Gardner",Alice's Adventures in Wonderland & Through the...,4.06,344482
2176,2119,319,0.878392,17245,2932,"Daniel Defoe, Virginia Woolf",Robinson Crusoe,3.66,181415
2176,2119,209,0.870232,17245,1953,"Charles Dickens, Richard Maxwell",A Tale of Two Cities,3.82,646983


#### Шаг 2. Сервис Feature Store**

Теперь сделаем так, чтобы набор стал доступен сервису рекомендаций. Для этого создадим новый сервис, который при запуске будет загружать набор похожих объектов из файла `"similar_items.parquet"` и отдавать список похожих объектов через метод `/similar_items`.

#### **Задание 2 из 6**

Gjkexbv работоспособный сервис, возвращающий список похожих объектов через метод `/similar_items`.

Сохраним код сервиса в файле `features_service.py`.

### Шаг 3. Сервис Event Store

#### Задание 3 из 6

In [47]:
# Сохранить взаимодействие для пользователя `user_id` с объектом `item_id`:

import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result) 

{'result': 'ok'}


In [49]:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result) 

{'events': [17245]}


In [ ]:
user_id = 1127794

In [52]:
# 1. Проверяем, что для пользователя нет событий
print("Шаг 1. Проверка начального состояния")
resp = requests.post(
    events_store_url + "/get",
    headers=headers,
    params={"user_id": user_id}
)
print(resp.json())  # ожидаем {"events": []}

Шаг 1. Проверка начального состояния
{'events': []}


In [53]:
# 2. Сохраняем четыре события: 18734992, 18734992, 7785, 4731479
print("\nШаг 2. Сохраняем события")

for item_id in [18734992, 18734992, 7785, 4731479]:
    resp = requests.post(
        events_store_url + "/put",
        headers=headers,
        params={"user_id": user_id, "item_id": item_id}
    )
    print(f"PUT user_id={user_id}, item_id={item_id} ->", resp.json())


Шаг 2. Сохраняем события
PUT user_id=1127794, item_id=18734992 -> {'result': 'ok'}
PUT user_id=1127794, item_id=18734992 -> {'result': 'ok'}
PUT user_id=1127794, item_id=7785 -> {'result': 'ok'}
PUT user_id=1127794, item_id=4731479 -> {'result': 'ok'}


In [54]:
# 3. Получаем последние три события
print("\nШаг 3. Получаем последние 3 события")
resp = requests.post(
    events_store_url + "/get",
    headers=headers,
    params={"user_id": user_id, "k": 3}
)
print(resp.json())


Шаг 3. Получаем последние 3 события
{'events': [4731479, 7785, 18734992]}


#### **Шаг 4. Доработка сервиса рекомендаций**

#### **Задание 4 из 6**

Дополним код нового метода так, чтобы для последнего события пользователя, если оно есть, возвращался список похожих объектов. Это и будут онлайн-рекомендации.

In [64]:
recommendations_url = "http://127.0.0.1:8000"
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

{'recs': [236183, 16566, 160317]}


Список пустой. Это ожидаемо, так как для пользователя в Event Store пока нет никаких событий, чтобы по ним получить онлайн-рекомендации.

Добавим событие:

In [56]:
params = {"user_id": 1291248, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params) 

И снова получим онлайн-рекомендации для пользователя `1291248`:

In [65]:
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

{'recs': [236183, 16566, 160317]}


#### **Шаг 5. Добавим разнообразия в онлайн-рекомендации**

#### Задание 5 из 6
Дополним новую версию реализации метода /recommendations_online так, чтобы онлайн-рекомендации возвращались для трёх последних событий.


Тест

In [67]:
user_id = 1291248
event_item_ids = [41899, 102868, 5472, 5907]

for event_item_id in event_item_ids:
    resp = requests.post(
        events_store_url + "/put",
        headers=headers,
        params={"user_id": user_id, "item_id": event_item_id},
    )

params = {"user_id": user_id, "k": 5}

resp = requests.post(
    recommendations_url + "/recommendations_online",
    headers=headers,
    params=params,
)
online_recs = resp.json()

print(online_recs)

{'recs': [16249388, 16207483, 17406142, 17181891, 16174197, 837039, 16161136, 40090, 10314376, 11295616]}


#### **Задание 6 из 6**

Доработаем код обновлённого метода `/recommendations` так, чтобы реализовать предложенную выше схему блендинга.

Тест

In [68]:
# Сначала сгенерируем онлайн-события:

user_id = 1291250
event_item_ids =  [7144, 16299, 5907, 18135]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id}) 

In [69]:
# Получим 10 рекомендаций каждого типа для данного пользователя:

params = {"user_id": 1291250, 'k': 10}
resp_offline = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
resp_online = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
resp_blended = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)

recs_offline = resp_offline.json()["recs"]
recs_online = resp_online.json()["recs"]
recs_blended = resp_blended.json()["recs"]

print(recs_offline)
print(recs_online)
print(recs_blended) 

[22557272, 29056083, 18007564, 18143977, 16096824, 3, 9460487, 38447, 15881, 11235712]
[16249388, 16207483, 17406142, 17181891, 16174197, 18214458, 18241159, 48795, 13613441, 16090636, 13450545, 101001, 860560, 846044, 860559, 631097, 142465, 142464, 862004, 631127, 178153, 48717, 77430, 438452, 142460, 243601, 46460, 19494, 18999, 91017]
[16249388, 22557272, 16207483, 29056083, 17406142, 18007564, 17181891, 18143977, 16174197, 16096824]


In [71]:
# Качество рекомендаций также можно оценить выборочно: посмотрев, что рекомендации книг в целом адекватны, по авторам, названиям.
# Пример кода для этого приведён ниже.

def display_items(item_ids):
    possible_cols = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    item_columns_to_use = [c for c in possible_cols if c in items.columns]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)
    
print("Онлайн-события")
display_items(event_item_ids)
print("Офлайн-рекомендации")
display_items(recs_offline)
print("Онлайн-рекомендации")
display_items(recs_online)
print("Рекомендации")
display_items(recs_blended) 

Онлайн-события


,item_id,author,title,average_rating,ratings_count
0,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,4.19,390293
1,16299,Agatha Christie,And Then There Were None,4.23,429352
2,5907,J.R.R. Tolkien,The Hobbit,4.25,2099680
3,18135,"William Shakespeare, Paul Werstine, Barbara A....",Romeo and Juliet,3.74,1656919


Офлайн-рекомендации


,item_id,author,title,average_rating,ratings_count
0,22557272,Paula Hawkins,The Girl on the Train,3.88,1076144
1,29056083,"John Tiffany, Jack Thorne, J.K. Rowling",Harry Potter and the Cursed Child - Parts One ...,3.74,288018
2,18007564,Andy Weir,The Martian,4.39,435440
3,18143977,Anthony Doerr,All the Light We Cannot See,4.31,498685
4,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,4.28,182581
5,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,4.45,4765497
6,9460487,Ransom Riggs,Miss Peregrine’s Home for Peculiar Children (M...,3.89,641884
7,38447,Margaret Atwood,The Handmaid's Tale,4.07,648783
8,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,4.38,1821802
9,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",4.15,441530


Онлайн-рекомендации


,item_id,author,title,average_rating,ratings_count
0,16249388,Ella Frank,"Edible (Exquisite, #3)",4.19,5925
1,16207483,"Ella Frank, MacKenzie Cartwright","Exquisite (Exquisite, #1)",3.96,10024
2,17406142,R.K. Lilley,Lana,3.92,11902
3,17181891,Nicole Edwards,"Kaleb (Alluring Indulgence, #1)",4.08,14361
4,16174197,Kitty French,"Knight & Play (Knight, #1)",4.03,24536
5,18214458,Avery Aster,"Unscrupulous (The Manhattanites, #1)",3.72,1253
6,18241159,Harper Sloan,"Axel (Corps Security, #1)",4.13,8619
7,48795,"Carolyn Keene, Walter Karig","Password to Larkspur Lane (Nancy Drew, #10)",3.93,8636
8,13613441,Elle Aycart,"Heavy Issues (Bowen Boys, #2)",4.08,8832
9,16090636,Kristen Proby,"Fight with Me (With Me in Seattle, #2)",4.25,39973


Рекомендации


,item_id,author,title,average_rating,ratings_count
0,16249388,Ella Frank,"Edible (Exquisite, #3)",4.19,5925
1,22557272,Paula Hawkins,The Girl on the Train,3.88,1076144
2,16207483,"Ella Frank, MacKenzie Cartwright","Exquisite (Exquisite, #1)",3.96,10024
3,29056083,"John Tiffany, Jack Thorne, J.K. Rowling",Harry Potter and the Cursed Child - Parts One ...,3.74,288018
4,17406142,R.K. Lilley,Lana,3.92,11902
5,18007564,Andy Weir,The Martian,4.39,435440
6,17181891,Nicole Edwards,"Kaleb (Alluring Indulgence, #1)",4.08,14361
7,18143977,Anthony Doerr,All the Light We Cannot See,4.31,498685
8,16174197,Kitty French,"Knight & Play (Knight, #1)",4.03,24536
9,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,4.28,182581
